In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import warnings
import os

# Suppress Keras warnings (optional)
warnings.filterwarnings('ignore')

# Load the trained model
model_path = r'model (1).h5'  # Default path
if not os.path.exists(model_path):
    print(f"Error: Model file not found at {model_path}")
    print("Please ensure 'model.h5' is in 'E:\\emotion_detection\\' or provide the correct path.")
    model_path = input("Enter the full path to your model.h5 file (e.g., E:\\other_folder\\model.h5): ").strip()
    if not os.path.exists(model_path):
        print(f"Error: Model file not found at {model_path}. Exiting.")
        exit()

try:
    model = load_model(model_path)
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    print("Ensure the file is a valid Keras .h5 model and compatible with your TensorFlow version.")
    exit()

# Define emotion labels (based on your model)
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Load the Haar cascade for face detection
face_cascade_path = r'haarcascade_frontalface_default.xml'
if not os.path.exists(face_cascade_path):
    print(f"Error: Haar cascade file not found at {face_cascade_path}")
    print("Please ensure 'haarcascade_frontalface_default.xml' is in 'E:\\emotion_detection\\'")
    print("Alternatively, use OpenCV's built-in cascade:")
    print("face_cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'")
    exit()

face_cascade = cv2.CascadeClassifier(face_cascade_path)
if face_cascade.empty():
    print(f"Error: Failed to load Haar cascade file at {face_cascade_path}")
    exit()

# Initialize webcam (index 0 worked based on your output)
cap = cv2.VideoCapture(0, cv2.CAP_ANY)
if not cap.isOpened():
    print("Error: Could not open webcam. Please check webcam connection, permissions, or drivers.")
    print("Try running the following to identify the correct index:")
    print("""
    import cv2
    for i in range(5):
        cap = cv2.VideoCapture(i)
        if cap.isOpened():
            print(f"Camera found at index {i}")
            cap.release()
    """)
    exit()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break

    # Convert frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Extract the region of interest (ROI) for the face
        roi = gray[y:y+h, x:x+w]
        
        # Resize the face ROI to 48x48 (assumed model input size)
        roi = cv2.resize(roi, (48, 48))
        
        # Convert to array and preprocess for the model
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)
        roi = roi / 255.0  # Normalize to [0,1]

        # Predict emotion
        try:
            predictions = model.predict(roi, verbose=0)[0]
            emotion = emotion_labels[np.argmax(predictions)]
            confidence = np.max(predictions)
        except Exception as e:
            print(f"Error during prediction: {e}")
            continue

        # Draw rectangle and label on the frame
        label = f"{emotion}: {confidence:.2f}"
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Emotion Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()

Model loaded successfully.


KeyboardInterrupt: 